In [1]:

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp


from biopsykit.classification.model_selection import SklearnPipelinePermuter

from pepbench.io import load_best_performing_algos_b_point, load_preprocessed_training_data, compute_abs_error, \
     get_best_pipeline_results, get_best_estimator, get_pipeline_steps, compute_mae_std_from_permuter

import pingouin as pg

import matplotlib.pyplot as plt

#%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
model_path = Path(f"../../../results/models")
data_path = Path(f"../../../results/data")
result_path = Path(f"../../../results")
rater = "rater_01"

In [3]:
permuter_include_nan = SklearnPipelinePermuter.from_pickle(model_path.joinpath(f'b-point/rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_rr_include_nan_rater_01.pkl'))
permuter_impute_nan = SklearnPipelinePermuter.from_pickle(model_path.joinpath(f'b-point/rr-interval/{rater}/b_point_regression_hpc_0_baseline_result_rr_median_imputed_rater_01.pkl'))
permuter_include_nan.metric_summary()

c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarn

conf_matrix conf_matrix_folds  \
pipeline_scaler pipeline_clf                                          
StandardScaler  DecisionTreeRegressor       [0.0]                []   
                RandomForestRegressor       [0.0]                []   
MinMaxScaler    DecisionTreeRegressor       [0.0]                []   
                RandomForestRegressor       [0.0]                []   

                                                                             true_labels  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                RandomForestRegressor  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
MinMaxScaler    DecisionTreeRegressor  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   
                RandomForestRegressor  [516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...   

                                                                       true_labels_folds  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...   
                RandomForestRegressor  [[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...   
MinMaxScaler    DecisionTreeRegressor  [[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...   
                RandomForestRegressor  [[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...   

                                                                        predicted_labels  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [524.4705882352941, 524.4705882352941, 498.121...   
                RandomForestRegressor  [526.4770836711158, 520.2811681308258, 494.300...   
MinMaxScaler    DecisionTreeRegressor  [527.5238095238095, 522.1904761904761, 500.090...   
                RandomForestRegressor  [525.699485398262, 520.0283937211624, 493.0129...   

                                                                  predicted_labels_folds  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [[524.4705882352941, 524.4705882352941, 498.12...   
                RandomForestRegressor  [[526.4770836711158, 520.2811681308258, 494.30...   
MinMaxScaler    DecisionTreeRegressor  [[527.5238095238095, 522.1904761904761, 500.09...   
                RandomForestRegressor  [[525.699485398262, 520.0283937211624, 493.012...   

                                                                           train_indices  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                RandomForestRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
MinMaxScaler    DecisionTreeRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                RandomForestRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                                                     train_indices_folds  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
                RandomForestRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
MinMaxScaler    DecisionTreeRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
                RandomForestRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   

                                                                            test_indices  \
pipeline_scaler pipeline_clf                                                               
StandardScaler  DecisionTreeRegressor  [321, 322, 323, 324, 325, 326, 327, 328, 329, ...   
                R

In [4]:
algo_dict_include_nan = {
    "MinMax_None_RFR_RR-Interval-Include-Nan": ('MinMaxScaler', 'RandomForestRegressor'),
    "SS_None_RFR_RR-Interval-Include-Nan": ('StandardScaler', 'RandomForestRegressor'),
}
algo_dict_impute_nan = {
    "SS_SFM_RFR_RR-Interval-Median-Imputed": ('StandardScaler', 'SelectFromModel', 'RandomForestRegressor'),
    "MinMax_KBest_RFR_RR-Interval-Median-Imputed": ('MinMaxScaler', 'SelectKBest', 'RandomForestRegressor'),
}

In [5]:
algo_include_nan = algo_dict_include_nan["SS_None_RFR_RR-Interval-Include-Nan"]

In [6]:
permuter_include_nan_summary = permuter_include_nan.metric_summary()
permuter_include_nan_summary[permuter_include_nan_summary.index == algo_include_nan]

,,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,train_indices,train_indices_folds,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4
pipeline_scaler,pipeline_clf,,,,,,,,,,,,,,,,,
StandardScaler,RandomForestRegressor,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[526.4770836711158, 520.2811681308258, 494.300...","[[526.4770836711158, 520.2811681308258, 494.30...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[[321, 322, 323, 324, 325, 326, 327, 328, 329,...",8.201392,1.114428,6.493249,8.559758,7.957382,8.040698,9.955874


In [7]:
pipeline_include_nan = permuter_include_nan.best_estimator_summary().loc[algo_include_nan]['best_estimator'].pipeline
pipeline_include_nan

[Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('clf',
                  RandomForestRegressor(ccp_alpha=0.01, max_features=0.8,
                                        min_impurity_decrease=0.01,
                                        min_samples_split=5, n_estimators=150,
                                        random_state=RandomState(MT19937) at 0x1EE23B17A40))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('clf',
                  RandomForestRegressor(ccp_alpha=0.01, criterion='friedman_mse',
                                        max_depth=np.int64(60), max_features=0.8,
                                        min_impurity_decrease=0.001,
                                        min_samples_split=5, n_estimators=200,
                                        random_state=RandomState(MT19937) at 0x1EE23B17C40))]),
 Pipeline(memory=Memory(locat

In [8]:
features_include_nan = np.array(pd.read_csv(data_path.joinpath(f'b-point/rr-interval/{rater}/train_data_b_point_rr_interval_include_nan.csv'), index_col=[0,1,2,3,4,5]).columns)
all_feature_importances = []
for fold in range(0,5,1):
    #selected_features = features_include_nan[pipeline_include_nan[fold]['reduce_dim'].get_support()]
    feature_importances = pipeline_include_nan[fold]['clf'].feature_importances_
    all_feature_importances.append(list(zip(features_include_nan, feature_importances)))
all_feature_importances = [
    [(key, float(value)) for key, value in inner_list]
    for inner_list in all_feature_importances
]
for fold, importances in enumerate(all_feature_importances):
    all_feature_importances[fold] = sorted(all_feature_importances[fold], key=lambda x: x[0], reverse=True)
all_feature_importances

[[('stern1985', 0.005052787738518284),
  ('sherwood1990', 0.00246683960008816),
  ('rr_interval_ms_estimated', 0.016718129161650226),
  ('pale2021', 0.0011235449153174442),
  ('miljkovic2022', 0.00311939535296303),
  ('lozano2007-quadratic-regression', 0.011746100564307155),
  ('lozano2007-linear-regression', 0.2015533951166521),
  ('forouzanfar2018', 0.007320597458056808),
  ('drost2022', 0.7174749115021225),
  ('debski1993-second-derivative', 0.027032459434892693),
  ('arbol2017-third-derivative', 0.002897825464565113),
  ('arbol2017-second-derivative', 0.002288614561271084),
  ('arbol2017-isoelectric-crossings', 0.001205399129595518)],
 [('stern1985', 0.005056657326486958),
  ('sherwood1990', 0.002201240459935289),
  ('rr_interval_ms_estimated', 0.007499299206199999),
  ('pale2021', 0.0009251067883879179),
  ('miljkovic2022', 0.0029167459437018867),
  ('lozano2007-quadratic-regression', 0.008047608638720923),
  ('lozano2007-linear-regression', 0.19814375413775964),
  ('forouzanfar20

In [9]:
# all feature importances has to be sorted by the feature name
averaged_feature_importances = list(zip([key for key, value in all_feature_importances[0]], np.mean([[value for key, value in fold] for fold in all_feature_importances], axis=0), np.std([[value for key, value in fold] for fold in all_feature_importances], axis=0)))
averaged_feature_importances = sorted(averaged_feature_importances, key=lambda x: x[1], reverse=True)
averaged_feature_importances

[('drost2022',
  np.float64(0.7148842932845543),
  np.float64(0.008707067514545011)),
 ('lozano2007-linear-regression',
  np.float64(0.20004226742867015),
  np.float64(0.01037037041875282)),
 ('debski1993-second-derivative',
  np.float64(0.039920287146538495),
  np.float64(0.01758180494645368)),
 ('rr_interval_ms_estimated',
  np.float64(0.011244196763921143),
  np.float64(0.0033253776993044425)),
 ('forouzanfar2018',
  np.float64(0.008960467441880098),
  np.float64(0.0014925729924553765)),
 ('lozano2007-quadratic-regression',
  np.float64(0.007497925682374175),
  np.float64(0.002698149280498188)),
 ('stern1985',
  np.float64(0.005661240604079109),
  np.float64(0.0005026116040352783)),
 ('miljkovic2022',
  np.float64(0.0028298837683938204),
  np.float64(0.0005092713079572367)),
 ('arbol2017-third-derivative',
  np.float64(0.0025694771941959706),
  np.float64(0.000284183535756846)),
 ('arbol2017-second-derivative',
  np.float64(0.0021685039047148248),
  np.float64(0.00027179538232115853

In [11]:
averaged_feature_importances_table = pd.DataFrame(averaged_feature_importances, columns=['feature', 'importance', 'std of importance'])
averaged_feature_importances_table["std in percent"] = averaged_feature_importances_table["std of importance"] / averaged_feature_importances_table["importance"] * 100
averaged_feature_importances_table

,feature,importance,std of importance,std in percent
0,drost2022,0.714884,0.008707,1.217969
1,lozano2007-linear-regression,0.200042,0.010370,5.184090
2,debski1993-second-derivative,0.039920,0.017582,44.042281
3,rr_interval_ms_estimated,0.011244,0.003325,29.574169
4,forouzanfar2018,0.008960,0.001493,16.657312
5,lozano2007-quadratic-regression,0.007498,0.002698,35.985276
6,stern1985,0.005661,0.000503,8.878118
7,miljkovic2022,0.002830,0.000509,17.996192
8,arbol2017-third-derivative,0.002569,0.000284,11.059975
9,arbol2017-second-derivative,0.002169,0.000272,12.533774


In [75]:
algo_impute_nan = algo_dict_impute_nan["SS_SFM_RFR_RR-Interval-Median-Imputed"]

In [76]:
permuter_impute_nan_summary = permuter_impute_nan.metric_summary()
permuter_impute_nan_summary[permuter_impute_nan_summary.index == algo_impute_nan]

,,,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,train_indices,train_indices_folds,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4
pipeline_scaler,pipeline_reduce_dim,pipeline_clf,,,,,,,,,,,,,,,,,
StandardScaler,SelectFromModel,RandomForestRegressor,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[529.9171036213073, 519.7807799163846, 490.494...","[[529.9171036213073, 519.7807799163846, 490.49...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[318, 319, 320, 321, 322, 323, 324, 325, 326, ...","[[318, 319, 320, 321, 322, 323, 324, 325, 326,...",8.316121,1.060872,7.380681,8.362111,9.262805,6.899659,9.675351


In [77]:
pipeline_impute_nan = permuter_impute_nan.best_estimator_summary().loc[algo_impute_nan]['best_estimator'].pipeline
pipeline_impute_nan

[Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  RandomForestRegressor(ccp_alpha=0.01, max_depth=np.int64(70),
                                        max_features=0.5, min_samples_split=10,
                                        n_estimators=400,
                                        random_state=RandomState(MT19937) at 0x19862DF5240))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  RandomForestRegressor(ccp_alpha=0.05, max_depth=np.int64(70),
                  

In [78]:
features_impute_nan = np.array(pd.read_csv(data_path.joinpath(f'b-point/rr-interval/{rater}/train_data_b_point_rr_interval_median_imputed.csv'), index_col=[0,1,2,3,4,5]).columns)

In [79]:
all_feature_importances = []
for fold in range(0,5,1):
    selected_features = features_impute_nan[pipeline_impute_nan[fold]['reduce_dim'].get_support()]
    feature_importances = pipeline_impute_nan[fold]['clf'].feature_importances_
    all_feature_importances.append(list(zip(selected_features, feature_importances)))
all_feature_importances = [
    [(key, float(value)) for key, value in inner_list]
    for inner_list in all_feature_importances
]
for fold, importances in enumerate(all_feature_importances):
    all_feature_importances[fold] = sorted(all_feature_importances[fold], key=lambda x: x[1], reverse=True)
all_feature_importances

[[('drost2022', 0.4212454246481978),
  ('lozano2007-linear-regression', 0.26358190040606294),
  ('debski1993-second-derivative', 0.164940885308496),
  ('forouzanfar2018', 0.08914262236370915),
  ('stern1985', 0.03660978424431202),
  ('rr_interval_ms_estimated', 0.019226105144718134),
  ('arbol2017-third-derivative', 0.0052532778845039265)],
 [('drost2022', 0.6665026557417916),
  ('lozano2007-linear-regression', 0.24210010372873095),
  ('debski1993-second-derivative', 0.05631198633458727),
  ('rr_interval_ms_estimated', 0.013464351253812871),
  ('forouzanfar2018', 0.010204063804423244),
  ('lozano2007-quadratic-regression', 0.00588311677661359),
  ('stern1985', 0.005533722360040373)],
 [('drost2022', 0.6742693411319618),
  ('lozano2007-linear-regression', 0.22200475244611623),
  ('debski1993-second-derivative', 0.047057925765229595),
  ('rr_interval_ms_estimated', 0.03141389773180351),
  ('forouzanfar2018', 0.014038469046199297),
  ('stern1985', 0.007076924642857587),
  ('arbol2017-seco